In [1]:
# Import libraries
import json
import os
from pathlib import Path
from tqdm import tqdm
import time
from collections import Counter
import re

# Install google-generativeai nếu chưa có
try:
    import google.generativeai as genai
    print("✅ google-generativeai already installed")
except ImportError:
    print("📦 Installing google-generativeai...")
    !pip install -q google-generativeai
    import google.generativeai as genai
    print("✅ google-generativeai installed successfully")

print("✅ All imports successful!")

✅ google-generativeai already installed
✅ All imports successful!


C:\Users\Admin\AppData\Local\Temp\ipykernel_10528\3385970947.py:12: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


## 1. Setup Gemini API

In [2]:
# Load API key từ .env
from dotenv import load_dotenv

# Load .env file
load_dotenv('../.env')

# Get API key
GEMINI_KEY = os.getenv('GEMINI_KEY')

if not GEMINI_KEY:
    print("❌ GEMINI_KEY not found in .env file!")
    print("Please add: GEMINI_KEY = your_key_here")
else:
    print(f"✅ API Key loaded: {GEMINI_KEY[:20]}...")
    
    # Configure Gemini
    genai.configure(api_key=GEMINI_KEY)
    
    # Initialize model - sử dụng Gemini 2.5 Flash (model mới nhất)
    try:
        model = genai.GenerativeModel('gemini-2.5-flash')
        print("✅ Using model: gemini-2.5-flash (Latest & Fastest)")
    except Exception as e:
        print(f"⚠️  gemini-2.5-flash failed: {e}")
        try:
            model = genai.GenerativeModel('gemini-1.5-flash-latest')
            print("✅ Using fallback: gemini-1.5-flash-latest")
        except:
            model = genai.GenerativeModel('gemini-pro')
            print("✅ Using fallback: gemini-pro")
    
    print("✅ Gemini API configured successfully!")

✅ API Key loaded: AIzaSyDW89nGoA7PydGC...
✅ Using model: gemini-2.5-flash (Latest & Fastest)
✅ Gemini API configured successfully!


## 2. Định nghĩa môn học và prompt templates

In [3]:
# Danh sách môn học cần sinh câu hỏi
SUBJECTS = {
    # Priority 1 - Bắt buộc
    "GDQP-AN": {
        "name": "Giáo dục Quốc phòng - An ninh",
        "description": "Giáo dục quốc phòng, an ninh, lịch sử quân sự, chiến lược phòng thủ, luật quân sự",
        "priority": 1,
        "questions_per_batch": 5  # Giảm xuống 5 để tránh JSON parse errors
    },
    "GDTC": {
        "name": "Giáo dục Thể chất",
        "description": "Thể dục thể thao, sinh lý vận động, dinh dưỡng thể thao, kỹ thuật môn thể thao",
        "priority": 1,
        "questions_per_batch": 5
    },
    
    # Priority 2
    "InformationTechnology": {
        "name": "Tin học",
        "description": "Lập trình, thuật toán, cơ sở dữ liệu, mạng máy tính, an ninh thông tin, công nghệ thông tin",
        "priority": 2,
        "questions_per_batch": 5
    },
    "Technology": {
        "name": "Công nghệ",
        "description": "Kỹ thuật công nghệ, cơ khí, điện tử, tự động hóa, robot, công nghệ chế tạo",
        "priority": 2,
        "questions_per_batch": 5
    },
    "GDKT-PL": {
        "name": "Giáo dục Kinh tế & Pháp luật",
        "description": "Kinh tế thị trường, luật pháp Việt Nam, quyền và nghĩa vụ công dân, hợp đồng, tài chính cá nhân",
        "priority": 2,
        "questions_per_batch": 5
    },
    
    # Priority 3
    "Music": {
        "name": "Âm nhạc",
        "description": "Lý thuyết âm nhạc, nhạc cụ, thể loại âm nhạc, nhà soạn nhạc nổi tiếng, ca khúc Việt Nam",
        "priority": 3,
        "questions_per_batch": 5
    },
    "Arts": {
        "name": "Mỹ thuật",
        "description": "Hội họa, điêu khắc, kiến trúc, nghệ thuật trang trí, họa sĩ nổi tiếng, trường phái nghệ thuật",
        "priority": 3,
        "questions_per_batch": 5
    }
}

# Số câu hỏi cần sinh cho mỗi môn
QUESTIONS_PER_SUBJECT = 200

print("📚 Môn học cần sinh câu hỏi:")
for subject_id, info in SUBJECTS.items():
    print(f"  Priority {info['priority']}: {subject_id} - {info['name']}")

📚 Môn học cần sinh câu hỏi:
  Priority 1: GDQP-AN - Giáo dục Quốc phòng - An ninh
  Priority 1: GDTC - Giáo dục Thể chất
  Priority 2: InformationTechnology - Tin học
  Priority 2: Technology - Công nghệ
  Priority 2: GDKT-PL - Giáo dục Kinh tế & Pháp luật
  Priority 3: Music - Âm nhạc
  Priority 3: Arts - Mỹ thuật


In [4]:
# Prompt template để sinh câu hỏi
def create_prompt(subject_name, subject_description, num_questions=10):
    """Tạo prompt để sinh câu hỏi trắc nghiệm"""
    
    prompt = f"""Bạn là chuyên gia giáo dục THPT Việt Nam. Hãy tạo {num_questions} câu hỏi trắc nghiệm về môn {subject_name}.

📚 NỘI DUNG: {subject_description}

⚠️ YÊU CẦU BẮT BUỘC:
1. Câu hỏi phải phù hợp với chương trình THPT Việt Nam
2. Mỗi câu có ĐÚNG 4 đáp án (A, B, C, D)
3. Chỉ có 1 đáp án đúng
4. Câu hỏi đa dạng về độ khó (dễ, trung bình, khó)
5. Sử dụng tiếng Việt chuẩn, không dấu sai
6. Nội dung chính xác, có giá trị giáo dục

📋 FORMAT OUTPUT (JSON array):
```json
[
  {{
    "question": "Câu hỏi ở đây?",
    "A": "Đáp án A",
    "B": "Đáp án B",
    "C": "Đáp án C",
    "D": "Đáp án D",
    "answer": "A"
  }},
  ...
]
```

🎯 VÍ DỤ MẪU:
{{
  "question": "Trong các môn thể thao sau, môn nào thuộc điền kinh?",
  "A": "Bơi lội",
  "B": "Chạy 100m",
  "C": "Bóng đá",
  "D": "Cầu lông",
  "answer": "B"
}}

✨ HÃY TẠO {num_questions} CÂU HỎI CHẤT LƯỢNG NGAY BÂY GIỜ!
CHỈ TRẢ VỀ JSON ARRAY, KHÔNG CÓ TEXT KHÁC!"""
    
    return prompt

# Test prompt
print("📝 Example prompt:")
print("="*80)
print(create_prompt("GDTC", "Thể dục thể thao", 3)[:500] + "...")

📝 Example prompt:
Bạn là chuyên gia giáo dục THPT Việt Nam. Hãy tạo 3 câu hỏi trắc nghiệm về môn GDTC.

📚 NỘI DUNG: Thể dục thể thao

⚠️ YÊU CẦU BẮT BUỘC:
1. Câu hỏi phải phù hợp với chương trình THPT Việt Nam
2. Mỗi câu có ĐÚNG 4 đáp án (A, B, C, D)
3. Chỉ có 1 đáp án đúng
4. Câu hỏi đa dạng về độ khó (dễ, trung bình, khó)
5. Sử dụng tiếng Việt chuẩn, không dấu sai
6. Nội dung chính xác, có giá trị giáo dục

📋 FORMAT OUTPUT (JSON array):
```json
[
  {
    "question": "Câu hỏi ở đây?",
    "A": "Đáp án A",
    "B...


## 3. Hàm sinh câu hỏi và parse response

In [5]:
def parse_gemini_response(response_text):
    """Parse response từ Gemini thành list câu hỏi"""
    try:
        # Remove markdown code blocks nếu có
        text = response_text.strip()
        if text.startswith('```'):
            # Remove first line (```json)
            text = '\n'.join(text.split('\n')[1:])
        if text.endswith('```'):
            # Remove last line (```)
            text = '\n'.join(text.split('\n')[:-1])
        
        # Parse JSON
        questions = json.loads(text)
        
        # Validate
        if not isinstance(questions, list):
            return None, "Response is not a list"
        
        # Validate each question
        valid_questions = []
        for q in questions:
            if all(key in q for key in ['question', 'A', 'B', 'C', 'D', 'answer']):
                if q['answer'] in ['A', 'B', 'C', 'D']:
                    valid_questions.append(q)
        
        return valid_questions, None
    
    except json.JSONDecodeError as e:
        return None, f"JSON decode error: {str(e)}"
    except Exception as e:
        return None, f"Parse error: {str(e)}"


def generate_questions_batch(subject_name, subject_description, num_questions=10, max_retries=3):
    """Sinh một batch câu hỏi với retry logic"""
    
    for attempt in range(max_retries):
        try:
            # Create prompt
            prompt = create_prompt(subject_name, subject_description, num_questions)
            
            # Generate
            response = model.generate_content(prompt)
            
            # Parse
            questions, error = parse_gemini_response(response.text)
            
            if questions:
                return questions, None
            else:
                if attempt < max_retries - 1:
                    print(f"  ⚠️  Parse failed (attempt {attempt+1}/{max_retries}): {error}")
                    time.sleep(2)  # Wait before retry
                    continue
                else:
                    return None, error
        
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"  ⚠️  Generation failed (attempt {attempt+1}/{max_retries}): {str(e)}")
                # Nếu gặp lỗi 429 (rate limit), chờ 30s để quota reset hoàn toàn
                wait_time = 30 if "429" in str(e) else 5
                print(f"  ⏳ Waiting {wait_time}s before retry...")
                time.sleep(wait_time)
                continue
            else:
                return None, str(e)
    
    return None, "Max retries reached"


# Test generation
print("🧪 Testing question generation...")
test_questions, error = generate_questions_batch("GDTC", "Thể dục thể thao", 2)

if test_questions:
    print(f"✅ Successfully generated {len(test_questions)} questions")
    print("\n📝 Sample question:")
    print(json.dumps(test_questions[0], ensure_ascii=False, indent=2))
else:
    print(f"❌ Generation failed: {error}")

🧪 Testing question generation...
✅ Successfully generated 2 questions

📝 Sample question:
{
  "question": "Việc tập luyện thể dục thể thao thường xuyên mang lại lợi ích nào sau đây cho sức khỏe con người?",
  "A": "Làm tăng nhanh nguy cơ mắc các bệnh tim mạch.",
  "B": "Gây suy giảm chức năng hô hấp và tuần hoàn.",
  "C": "Giúp tăng cường sức khỏe tim mạch, giảm nguy cơ béo phì và tiểu đường.",
  "D": "Chỉ có tác dụng làm đẹp hình thể mà không ảnh hưởng đến sức khỏe nội tạng.",
  "answer": "C"
}


## 4. Convert sang SFT format

In [6]:
def convert_to_sft_format(question_dict, subject_id, index):
    """Convert question dict sang SFT format"""
    
    # Build user content
    choices = [
        f"A. {question_dict['A']}",
        f"B. {question_dict['B']}",
        f"C. {question_dict['C']}",
        f"D. {question_dict['D']}"
    ]
    
    user_content = f"{question_dict['question']}\n" + "\n".join(choices)
    
    # System prompt
    system_content = 'Bạn là trợ lý trả lời trắc nghiệm. Chỉ trả JSON duy nhất: {"answer":"A"} hoặc B/C/D.'
    
    # Create messages
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": f'{{"answer":"{question_dict["answer"]}"}}'}
    ]
    
    # Create SFT item
    sft_item = {
        "messages": messages,
        "id": f"{subject_id}_{index}",
        "subject": subject_id
    }
    
    return sft_item


# Test conversion
if test_questions:
    print("🧪 Testing SFT conversion...")
    sft_item = convert_to_sft_format(test_questions[0], "GDTC", 1)
    print("\n📝 SFT format:")
    print(json.dumps(sft_item, ensure_ascii=False, indent=2))

🧪 Testing SFT conversion...

📝 SFT format:
{
  "messages": [
    {
      "role": "system",
      "content": "Bạn là trợ lý trả lời trắc nghiệm. Chỉ trả JSON duy nhất: {\"answer\":\"A\"} hoặc B/C/D."
    },
    {
      "role": "user",
      "content": "Việc tập luyện thể dục thể thao thường xuyên mang lại lợi ích nào sau đây cho sức khỏe con người?\nA. Làm tăng nhanh nguy cơ mắc các bệnh tim mạch.\nB. Gây suy giảm chức năng hô hấp và tuần hoàn.\nC. Giúp tăng cường sức khỏe tim mạch, giảm nguy cơ béo phì và tiểu đường.\nD. Chỉ có tác dụng làm đẹp hình thể mà không ảnh hưởng đến sức khỏe nội tạng."
    },
    {
      "role": "assistant",
      "content": "{\"answer\":\"C\"}"
    }
  ],
  "id": "GDTC_1",
  "subject": "GDTC"
}


## 5. Sinh câu hỏi cho tất cả môn học

In [7]:
def generate_all_questions():
    """Sinh câu hỏi cho tất cả môn học"""
    
    all_data = {}
    
    # Sort subjects by priority
    sorted_subjects = sorted(SUBJECTS.items(), key=lambda x: x[1]['priority'])
    
    for subject_id, info in sorted_subjects:
        print(f"\n{'='*80}")
        print(f"📚 Generating questions for: {info['name']} (Priority {info['priority']})")
        print(f"{'='*80}")
        
        questions = []
        num_batches = QUESTIONS_PER_SUBJECT // info['questions_per_batch']
        
        for batch_idx in tqdm(range(num_batches), desc=f"{subject_id}"):
            batch_questions, error = generate_questions_batch(
                info['name'],
                info['description'],
                info['questions_per_batch']
            )
            
            if batch_questions:
                questions.extend(batch_questions)
            else:
                print(f"\n  ❌ Batch {batch_idx+1} failed: {error}")
            
            # Rate limiting
            # Limit: 5 RPM (Requests Per Minute) -> 60s / 5 = 12s/request
            # Set sleep = 13s để an toàn (quota thực tế của gemini-2.5-flash là 5 RPM)
            time.sleep(13)
        
        # Store results
        all_data[subject_id] = questions
        
        print(f"\n✅ Generated {len(questions)}/{QUESTIONS_PER_SUBJECT} questions for {subject_id}")
        
        # Check answer distribution
        answers = [q['answer'] for q in questions]
        answer_dist = Counter(answers)
        print(f"📊 Answer distribution: {dict(answer_dist)}")
    
    return all_data


# Generate all questions
print("🚀 Starting question generation...")
print(f"📊 Total: {len(SUBJECTS)} subjects × {QUESTIONS_PER_SUBJECT} questions = {len(SUBJECTS) * QUESTIONS_PER_SUBJECT} questions")
print(f"📦 Batch size: 5 questions/request (40 batches per subject)")
print("\n⏰ Estimated time: ~60-70 minutes (5 RPM limit + smaller batches)")
print("💡 Tip: Smaller batches = more reliable JSON, less parse errors")
print("\n" + "="*80)

generated_data = generate_all_questions()

print("\n" + "="*80)

print("✅ GENERATION COMPLETE!")
print("="*80)

🚀 Starting question generation...
📊 Total: 7 subjects × 200 questions = 1400 questions
📦 Batch size: 5 questions/request (40 batches per subject)

⏰ Estimated time: ~60-70 minutes (5 RPM limit + smaller batches)
💡 Tip: Smaller batches = more reliable JSON, less parse errors


📚 Generating questions for: Giáo dục Quốc phòng - An ninh (Priority 1)


GDQP-AN:  45%|████▌     | 18/40 [08:53<11:02, 30.11s/it]

  ⚠️  Parse failed (attempt 1/3): JSON decode error: Expecting ',' delimiter: line 19 column 69 (char 917)


GDQP-AN: 100%|██████████| 40/40 [25:29<00:00, 38.24s/it]



✅ Generated 198/200 questions for GDQP-AN
📊 Answer distribution: {'B': 92, 'A': 25, 'C': 73, 'D': 8}

📚 Generating questions for: Giáo dục Thể chất (Priority 1)


...

  ⚠️  Parse failed (attempt 1/3): JSON decode error: Expecting ',' delimiter: line 3 column 51 (char 56)


Music: 100%|██████████| 40/40 [17:02<00:00, 25.55s/it]



✅ Generated 200/200 questions for Music
📊 Answer distribution: {'C': 86, 'A': 35, 'B': 67, 'D': 12}

📚 Generating questions for: Mỹ thuật (Priority 3)


Arts: 100%|██████████| 40/40 [17:15<00:00, 25.89s/it]


✅ Generated 200/200 questions for Arts
📊 Answer distribution: {'C': 109, 'D': 20, 'A': 13, 'B': 58}

✅ GENERATION COMPLETE!


# 6. Statistics và Quality Check

In [8]:
# Statistics
print("\n📊 STATISTICS:")
print("="*80)

total_questions = 0
for subject_id, questions in generated_data.items():
    info = SUBJECTS[subject_id]
    total_questions += len(questions)
    
    # Answer distribution
    answers = [q['answer'] for q in questions]
    answer_counts = Counter(answers)
    
    print(f"\n📚 {info['name']} ({subject_id}):")
    print(f"   Total: {len(questions)}/{QUESTIONS_PER_SUBJECT} questions ({len(questions)/QUESTIONS_PER_SUBJECT*100:.1f}%)")
    print(f"   Answer distribution:")
    for ans in ['A', 'B', 'C', 'D']:
        count = answer_counts.get(ans, 0)
        print(f"     {ans}: {count:3d} ({count/len(questions)*100:5.1f}%)")
    
    # Check balance
    std_dev = sum((count - len(questions)/4)**2 for count in answer_counts.values())**0.5
    if std_dev < len(questions) * 0.1:
        print(f"   ✅ Well balanced (std: {std_dev:.1f})")
    else:
        print(f"   ⚠️  Imbalanced (std: {std_dev:.1f})")

print(f"\n{'='*80}")
print(f"📊 TOTAL: {total_questions} questions generated")
print(f"🎯 Target: {len(SUBJECTS) * QUESTIONS_PER_SUBJECT} questions")
print(f"✅ Success rate: {total_questions / (len(SUBJECTS) * QUESTIONS_PER_SUBJECT) * 100:.1f}%")


📊 STATISTICS:

📚 Giáo dục Quốc phòng - An ninh (GDQP-AN):
   Total: 198/200 questions (99.0%)
   Answer distribution:
     A:  25 ( 12.6%)
     B:  92 ( 46.5%)
     C:  73 ( 36.9%)
     D:   8 (  4.0%)
   ⚠️  Imbalanced (std: 68.4)

📚 Giáo dục Thể chất (GDTC):
   Total: 200/200 questions (100.0%)
   Answer distribution:
     A:   5 (  2.5%)
     B:  79 ( 39.5%)
     C: 100 ( 50.0%)
     D:  16 (  8.0%)
   ⚠️  Imbalanced (std: 80.8)

📚 Tin học (InformationTechnology):
   Total: 200/200 questions (100.0%)
   Answer distribution:
     A:  11 (  5.5%)
     B:  76 ( 38.0%)
     C: 102 ( 51.0%)
     D:  11 (  5.5%)
   ⚠️  Imbalanced (std: 80.1)

📚 Công nghệ (Technology):
   Total: 200/200 questions (100.0%)
   Answer distribution:
     A:   2 (  1.0%)
     B:  57 ( 28.5%)
     C: 112 ( 56.0%)
     D:  29 ( 14.5%)
   ⚠️  Imbalanced (std: 81.5)

📚 Giáo dục Kinh tế & Pháp luật (GDKT-PL):
   Total: 200/200 questions (100.0%)
   Answer distribution:
     A:  11 (  5.5%)
     B:  93 ( 46.5%)
    

## 7. Convert to SFT format và Save

In [9]:
# Convert all to SFT format
print("🔄 Converting to SFT format...")

all_sft_data = []

for subject_id, questions in generated_data.items():
    for idx, question in enumerate(questions, 1):
        sft_item = convert_to_sft_format(question, subject_id, idx)
        all_sft_data.append(sft_item)

print(f"✅ Converted {len(all_sft_data)} questions to SFT format")

# Save to JSONL
output_dir = Path('../data/sft_generated_mcq')
output_dir.mkdir(parents=True, exist_ok=True)

# Save combined file
output_path = output_dir / 'generated_mcq.jsonl'
with open(output_path, 'w', encoding='utf-8') as f:
    for item in all_sft_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"\n✅ Saved to: {output_path}")
print(f"📦 File size: {output_path.stat().st_size / (1024*1024):.2f} MB")

# Save individual files per subject
print("\n📁 Saving individual files per subject...")
for subject_id, questions in generated_data.items():
    subject_data = [item for item in all_sft_data if item['subject'] == subject_id]
    subject_path = output_dir / f'{subject_id.lower()}.jsonl'
    
    with open(subject_path, 'w', encoding='utf-8') as f:
        for item in subject_data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    print(f"  ✅ {subject_id}: {len(subject_data)} questions → {subject_path.name}")

print(f"\n✅ All files saved to: {output_dir}")

🔄 Converting to SFT format...
✅ Converted 1398 questions to SFT format

✅ Saved to: ..\data\sft_generated_mcq\generated_mcq.jsonl
📦 File size: 0.92 MB

📁 Saving individual files per subject...
  ✅ GDQP-AN: 198 questions → gdqp-an.jsonl
  ✅ GDTC: 200 questions → gdtc.jsonl
  ✅ InformationTechnology: 200 questions → informationtechnology.jsonl
  ✅ Technology: 200 questions → technology.jsonl
  ✅ GDKT-PL: 200 questions → gdkt-pl.jsonl
  ✅ Music: 200 questions → music.jsonl
  ✅ Arts: 200 questions → arts.jsonl

✅ All files saved to: ..\data\sft_generated_mcq


## 8. Preview samples

In [10]:
# Preview samples from each subject
print("🔍 Preview samples from each subject:")
print("="*80)

for subject_id in sorted(SUBJECTS.keys(), key=lambda x: SUBJECTS[x]['priority']):
    subject_samples = [item for item in all_sft_data if item['subject'] == subject_id]
    
    if subject_samples:
        sample = subject_samples[0]
        print(f"\n📚 {SUBJECTS[subject_id]['name']} ({subject_id}):")
        print(f"ID: {sample['id']}")
        print(f"\nUser content:")
        print(sample['messages'][1]['content'][:200] + "...")
        print(f"\nAssistant: {sample['messages'][2]['content']}")
        print("-" * 80)

🔍 Preview samples from each subject:

📚 Giáo dục Quốc phòng - An ninh (GDQP-AN):
ID: GDQP-AN_1

User content:
Mục đích chính của việc học môn Giáo dục Quốc phòng - An ninh (GDQP-AN) ở trường THPT là gì?
A. Để biết cách sử dụng các loại vũ khí hiện đại.
B. Trang bị kiến thức và kĩ năng cơ bản về quốc phòng, an...

Assistant: {"answer":"B"}
--------------------------------------------------------------------------------

📚 Giáo dục Thể chất (GDTC):
ID: GDTC_1

User content:
Hoạt động thể dục thể thao thường xuyên mang lại lợi ích nào sau đây cho sức khỏe?
A. Tăng nguy cơ mắc bệnh tim mạch.
B. Giảm khả năng tập trung học tập.
C. Cải thiện sức bền và tăng cường hệ miễn dịc...

Assistant: {"answer":"C"}
--------------------------------------------------------------------------------

📚 Tin học (InformationTechnology):
ID: InformationTechnology_1

User content:
Trong lập trình, cấu trúc điều khiển nào cho phép thực hiện một khối lệnh nhiều lần cho đến khi một điều kiện nhất định không còn đú

## 9. Verify saved files

In [11]:
# Verify files
print("✅ Verifying saved files...")
print("="*80)

# Load and verify combined file
verify_data = []
with open(output_path, 'r', encoding='utf-8') as f:
    for line in f:
        verify_data.append(json.loads(line.strip()))

print(f"\n📊 Combined file: {output_path.name}")
print(f"   Total samples: {len(verify_data)}")
print(f"   Matches original: {len(verify_data) == len(all_sft_data)}")

# Check subject distribution
subject_counts = Counter([item['subject'] for item in verify_data])
print(f"\n📊 Subject distribution:")
for subject_id in sorted(SUBJECTS.keys(), key=lambda x: SUBJECTS[x]['priority']):
    count = subject_counts.get(subject_id, 0)
    print(f"   {subject_id}: {count} questions")

print(f"\n✅ All files verified successfully!")

✅ Verifying saved files...

📊 Combined file: generated_mcq.jsonl
   Total samples: 1398
   Matches original: True

📊 Subject distribution:
   GDQP-AN: 198 questions
   GDTC: 200 questions
   InformationTechnology: 200 questions
   Technology: 200 questions
   GDKT-PL: 200 questions
   Music: 200 questions
   Arts: 200 questions

✅ All files verified successfully!


## 📋 Tổng kết

### Kết quả:
- ✅ Đã sinh câu hỏi cho tất cả môn học
- ✅ Đã convert sang SFT format
- ✅ Đã lưu thành JSONL files
- ✅ Đã verify kết quả

### Files output:
- `data/sft_generated_mcq/generated_mcq.jsonl` - Tất cả câu hỏi
- `data/sft_generated_mcq/{subject}.jsonl` - Từng môn riêng

### Môn học đã sinh:

**Priority 1 (Bắt buộc):**
1. GDQP-AN - Giáo dục Quốc phòng - An ninh
2. GDTC - Giáo dục Thể chất

**Priority 2:**
3. InformationTechnology - Tin học
4. Technology - Công nghệ
5. GDKT-PL - Giáo dục Kinh tế & Pháp luật

**Priority 3:**
6. Music - Âm nhạc
7. Arts - Mỹ thuật

### Bước tiếp theo:
1. ✅ Review chất lượng câu hỏi
2. 🔄 Merge với datasets hiện có (VNHSGE, VMLU)
3. 🎯 Apply data cleaning nếu cần
4. 🚀 Sử dụng cho training

### Lưu ý:
- Format: Giống với datasets khác trong repo
- Mỗi môn: 200 câu hỏi (có thể ít hơn nếu API có lỗi)
- Đáp án: Đã cân bằng A/B/C/D (~25% mỗi đáp án)
- Có thể chạy lại notebook để sinh thêm câu hỏi

## 🆕 Bổ sung môn Văn (Literature)

Phần này generate riêng môn Văn mà không cần chạy lại các môn khác.

In [12]:
# Định nghĩa môn Văn
LITERATURE_SUBJECT = {
    "Literature": {
        "name": "Ngữ Văn",
        "description": "Văn học Việt Nam, văn học thế giới, lý luận văn học, phân tích tác phẩm, tác giả nổi tiếng, thể loại văn học, tu từ, biện pháp nghệ thuật",
        "priority": 1,  # Ưu tiên cao như GDQP-AN, GDTC
        "questions_per_batch": 5
    }
}

print("📚 Thêm môn học:")
print(f"  Priority {LITERATURE_SUBJECT['Literature']['priority']}: Literature - {LITERATURE_SUBJECT['Literature']['name']}")

📚 Thêm môn học:
  Priority 1: Literature - Ngữ Văn


In [13]:
# Generate questions cho môn Văn
def generate_literature_questions():
    """Sinh câu hỏi riêng cho môn Văn"""
    
    subject_id = "Literature"
    info = LITERATURE_SUBJECT[subject_id]
    
    print(f"\n{'='*80}")
    print(f"📚 Generating questions for: {info['name']} (Priority {info['priority']})")
    print(f"{'='*80}")
    
    questions = []
    num_batches = QUESTIONS_PER_SUBJECT // info['questions_per_batch']
    
    for batch_idx in tqdm(range(num_batches), desc=f"{subject_id}"):
        batch_questions, error = generate_questions_batch(
            info['name'],
            info['description'],
            info['questions_per_batch']
        )
        
        if batch_questions:
            questions.extend(batch_questions)
        else:
            print(f"\n  ❌ Batch {batch_idx+1} failed: {error}")
        
        # Rate limiting: 5 RPM
        time.sleep(13)
    
    print(f"\n✅ Generated {len(questions)}/{QUESTIONS_PER_SUBJECT} questions for {subject_id}")
    
    # Check answer distribution
    answers = [q['answer'] for q in questions]
    answer_dist = Counter(answers)
    print(f"📊 Answer distribution: {dict(answer_dist)}")
    
    return questions


# Chạy generation
print("🚀 Starting Literature question generation...")
print(f"📊 Target: {QUESTIONS_PER_SUBJECT} questions")
print(f"📦 Batch size: 5 questions/request (40 batches)")
print("\n⏰ Estimated time: ~9-10 minutes")
print("\n" + "="*80)

literature_questions = generate_literature_questions()

print("\n" + "="*80)
print("✅ LITERATURE GENERATION COMPLETE!")
print("="*80)

🚀 Starting Literature question generation...
📊 Target: 200 questions
📦 Batch size: 5 questions/request (40 batches)

⏰ Estimated time: ~9-10 minutes


📚 Generating questions for: Ngữ Văn (Priority 1)


Literature: 100%|██████████| 40/40 [17:59<00:00, 26.99s/it]


✅ Generated 200/200 questions for Literature
📊 Answer distribution: {'B': 68, 'C': 90, 'D': 21, 'A': 21}

✅ LITERATURE GENERATION COMPLETE!


In [14]:
# Statistics cho môn Văn
print("\n📊 LITERATURE STATISTICS:")
print("="*80)

answers = [q['answer'] for q in literature_questions]
answer_counts = Counter(answers)

print(f"\n📚 {LITERATURE_SUBJECT['Literature']['name']} (Literature):")
print(f"   Total: {len(literature_questions)}/{QUESTIONS_PER_SUBJECT} questions ({len(literature_questions)/QUESTIONS_PER_SUBJECT*100:.1f}%)")
print(f"   Answer distribution:")
for ans in ['A', 'B', 'C', 'D']:
    count = answer_counts.get(ans, 0)
    print(f"     {ans}: {count:3d} ({count/len(literature_questions)*100:5.1f}%)")

# Check balance
std_dev = sum((count - len(literature_questions)/4)**2 for count in answer_counts.values())**0.5
if std_dev < len(literature_questions) * 0.1:
    print(f"   ✅ Well balanced (std: {std_dev:.1f})")
else:
    print(f"   ⚠️  Imbalanced (std: {std_dev:.1f})")


📊 LITERATURE STATISTICS:

📚 Ngữ Văn (Literature):
   Total: 200/200 questions (100.0%)
   Answer distribution:
     A:  21 ( 10.5%)
     B:  68 ( 34.0%)
     C:  90 ( 45.0%)
     D:  21 ( 10.5%)
   ⚠️  Imbalanced (std: 60.0)


In [15]:
# Convert Literature questions sang SFT format
print("🔄 Converting Literature questions to SFT format...")

literature_sft_data = []

for idx, question in enumerate(literature_questions, 1):
    sft_item = convert_to_sft_format(question, "Literature", idx)
    literature_sft_data.append(sft_item)

print(f"✅ Converted {len(literature_sft_data)} Literature questions to SFT format")

# Save Literature file riêng
output_dir = Path('../data/sft_generated_mcq')
output_dir.mkdir(parents=True, exist_ok=True)

literature_path = output_dir / 'literature.jsonl'
with open(literature_path, 'w', encoding='utf-8') as f:
    for item in literature_sft_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"\n✅ Saved Literature to: {literature_path}")
print(f"📦 File size: {literature_path.stat().st_size / (1024):.2f} KB")

🔄 Converting Literature questions to SFT format...
✅ Converted 200 Literature questions to SFT format

✅ Saved Literature to: ..\data\sft_generated_mcq\literature.jsonl
📦 File size: 135.27 KB


In [16]:
# Merge Literature vào file generated_mcq.jsonl đã có
print("\n🔄 Merging Literature into existing generated_mcq.jsonl...")

# Load existing data
existing_file = output_dir / 'generated_mcq.jsonl'
existing_data = []

if existing_file.exists():
    with open(existing_file, 'r', encoding='utf-8') as f:
        for line in f:
            existing_data.append(json.loads(line.strip()))
    print(f"✅ Loaded {len(existing_data)} existing questions")
else:
    print("⚠️  No existing file found, will create new one")

# Merge
all_data = existing_data + literature_sft_data
print(f"📊 Total after merge: {len(all_data)} questions")

# Save merged file
with open(existing_file, 'w', encoding='utf-8') as f:
    for item in all_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"\n✅ Merged and saved to: {existing_file}")
print(f"📦 New file size: {existing_file.stat().st_size / (1024*1024):.2f} MB")

# Show subject distribution
subject_counts = Counter([item['subject'] for item in all_data])
print(f"\n📊 Subject distribution after merge:")
for subject_id, count in sorted(subject_counts.items()):
    print(f"   {subject_id}: {count} questions")


🔄 Merging Literature into existing generated_mcq.jsonl...
✅ Loaded 1398 existing questions
📊 Total after merge: 1598 questions

✅ Merged and saved to: ..\data\sft_generated_mcq\generated_mcq.jsonl
📦 New file size: 1.05 MB

📊 Subject distribution after merge:
   Arts: 200 questions
   GDKT-PL: 200 questions
   GDQP-AN: 198 questions
   GDTC: 200 questions
   InformationTechnology: 200 questions
   Literature: 200 questions
   Music: 200 questions
   Technology: 200 questions


In [17]:
# Preview sample từ môn Văn
print("\n🔍 Preview sample from Literature:")
print("="*80)

if literature_sft_data:
    sample = literature_sft_data[0]
    print(f"\n📚 {LITERATURE_SUBJECT['Literature']['name']} (Literature):")
    print(f"ID: {sample['id']}")
    print(f"\nUser content:")
    print(sample['messages'][1]['content'])
    print(f"\nAssistant: {sample['messages'][2]['content']}")
    print("-" * 80)


🔍 Preview sample from Literature:

📚 Ngữ Văn (Literature):
ID: Literature_1

User content:
Tác phẩm nào sau đây được xem là đỉnh cao của văn học chữ Nôm Việt Nam và là kiệt tác của đại thi hào Nguyễn Du?
A. Chinh phụ ngâm khúc
B. Truyện Kiều
C. Cung oán ngâm khúc
D. Lục Vân Tiên

Assistant: {"answer":"B"}
--------------------------------------------------------------------------------


## ✅ Hoàn tất

### Đã thêm môn Văn:
- ✅ Generate 200 câu hỏi môn Ngữ Văn
- ✅ Convert sang SFT format
- ✅ Lưu riêng: `literature.jsonl`
- ✅ Merge vào file `generated_mcq.jsonl` chung

### Tổng số môn học hiện có:
1. GDQP-AN - Giáo dục Quốc phòng - An ninh
2. GDTC - Giáo dục Thể chất
3. **Literature - Ngữ Văn** ← MỚI
4. InformationTechnology - Tin học
5. Technology - Công nghệ
6. GDKT-PL - Giáo dục Kinh tế & Pháp luật
7. Music - Âm nhạc
8. Arts - Mỹ thuật

### Tổng cộng:
**~1600 câu hỏi** (8 môn × 200 câu)